In [6]:
import pandas as pd
import requests

## Data collection

Se realiza la lectura de los objetos directamente desde el github de felipeares.

In [7]:
url = 'https://raw.githubusercontent.com/felipeares/legalbot-desafio-objetos/master/objetos.txt'
page = requests.get(url)
#Se identificó visualmente que el separador de los objetos corresponde a los caracteres \n\n
df_corpus = pd.DataFrame(page.text.split('\n\n'), columns = ['objeto'])

In [8]:
df_corpus.head()

,objeto
0,"CONSTRUCCIÓN, ALQUILER DE OTROS TIPOS DE MAQUI..."
1,el servicio de turismo por todo el territorio ...
2,SALON DE BELLEZA.
3,"CURSOS ONLINE DE MUSICA, COCINA, CLASES DE MUS..."
4,"el ejercicio de la actividad comercial,importa..."


## Data Preprocessing

In [31]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Jota\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jota\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Jota\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [37]:
import string  #Se obtendrá los signos de puntuación
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer,SnowballStemmer


In [41]:
#Método limpieza inicial y tokenización
def get_tokens_clean(objeto):
    #Se deja el texto en minúscula
    objeto = objeto.lower()
    #Elimina tildes y diéresis
    filtered_punctuation =  objeto.translate({ord(c): None for c in string.punctuation})
    #tokeniza
    tokens = nltk.word_tokenize(filtered_punctuation)
    #Se eliminan stopwords (articulos, preposiciones, etc.)
    no_stopwords = [word for word in tokens if word not in stopwords.words('spanish')]
    
    return no_stopwords

#Función de derivacion 
def stem_words(tokens):
    stemmer_spanish = SnowballStemmer('spanish')
    stems = []
    for word in tokens:
        stem = stemmer_spanish.stem(word)
        stems.append(stem)
    return stems

In [47]:
df_corpus['objeto_tokens'] = df_corpus['objeto'].map(get_tokens_clean)

In [49]:
df_corpus.sample(10)

,objeto,objeto_tokens
435,"El objeto será la Construccion, contratista Ob...","[objeto, construccion, contratista, obras, inm..."
607,"Transporte urbano, interurbano, nacional e int...","[transporte, urbano, interurbano, nacional, in..."
1928,"A) la Prestación de Servicios de Estudio, eval...","[prestación, servicios, estudio, evaluación, p..."
58,"Contratista en construcción (Carpintería, Alba...","[contratista, construcción, carpintería, albañ..."
2313,ALQUILER DE OTROS TIPOS DE MAQUINARIA Y EQUIPO...,"[alquiler, tipos, maquinaria, equipo, alquiler..."
2317,DETECCION Y REVISION DE FALLAS DE VEHICULOS AU...,"[deteccion, revision, fallas, vehiculos, autom..."
1682,la inversión y explotación de recursos en toda...,"[inversión, explotación, recursos, toda, clase..."
660,Venta de insumos mineros. Suministro de insumo...,"[venta, insumos, mineros, suministro, insumos,..."
578,SERVICIOS PERSONALES DE EDUCACION Y OTROS SERV...,"[servicios, personales, educacion, servicios, ..."
2014,"PRODUCCION DE EVENTOS, ACTIVIDADES PUBLICITARI...","[produccion, eventos, actividades, publicitari..."
